In [2]:
# Instalar pacotes necessários
!pip install biopython scikit-learn imbalanced-learn
from Bio import SeqIO
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.1 MB/s eta 0:00:00


In [4]:
# Ler o arquivo de treino
train_sequences = list(SeqIO.parse("modified_perfect_transposons.fasta", "fasta"))
train_data = [str(seq.seq) for seq in train_sequences]
train_labels = [seq.id for seq in train_sequences]

# Vetorização das sequências usando TF-IDF
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 4))
X_train = vectorizer.fit_transform(train_data)

# Dividir os dados de treino em treino e validação
X_train_split, X_val, y_train, y_val = train_test_split(X_train, train_labels, test_size=0.2, random_state=42)

# Definição de modelos para teste
models = {
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(kernel='rbf', probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

In [6]:
# Testar cada modelo
for name, model in models.items():
    # Treinar o modelo
    model.fit(X_train_split, y_train)

    # Avaliar o modelo
    y_pred = model.predict(X_val)
    print(f"\nModelo: {name}")
    print(classification_report(y_val, y_pred))

# Exemplo de tuning de hiperparâmetros para SVM
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['rbf', 'poly']}
grid = GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=3)
grid.fit(X_train_split, y_train)

# Avaliar o melhor modelo encontrado
y_pred = grid.predict(X_val)
print("\nmodelo SVM após tuning:")
print(classification_report(y_val, y_pred))


Modelo: Naive Bayes
                 precision    recall  f1-score   support

 DNA-transposon       0.92      0.26      0.40       172
Retrotransposon       0.58      0.98      0.73       182

       accuracy                           0.63       354
      macro avg       0.75      0.62      0.56       354
   weighted avg       0.74      0.63      0.57       354


Modelo: SVM
                 precision    recall  f1-score   support

 DNA-transposon       0.85      0.62      0.72       172
Retrotransposon       0.71      0.90      0.80       182

       accuracy                           0.76       354
      macro avg       0.78      0.76      0.76       354
   weighted avg       0.78      0.76      0.76       354


Modelo: Random Forest
                 precision    recall  f1-score   support

 DNA-transposon       0.88      0.89      0.88       172
Retrotransposon       0.89      0.88      0.89       182

       accuracy                           0.89       354
      macro avg       0

In [23]:
# Prever as classes para o conjunto de teste utilizando SVM
test_sequences = list(SeqIO.parse("reduced.fa", "fasta"))
test_data = [str(seq.seq) for seq in test_sequences]
test_ids = [seq.id for seq in test_sequences]
X_test = vectorizer.transform(test_data)

predictions = grid.predict(X_test)
results = dict(zip(test_ids, predictions))

# Exibir as previsões
#for seq_id, prediction in results.items():
#    print(f"ID: {seq_id}, Predição: {prediction}")
# Salvar as previsões em um arquivo
with open('PrevisionsSvmTun.txt', 'w') as f:
    for seq_id, prediction in results.items():
        f.write(f"{seq_id}\t{prediction}\n")

# Baixar via navegador o arquivo de resultados
#from google.colab import files
#files.download('PrevisionsSvmTun.txt')

In [24]:
# predições com Randon Forest
predictions = models['Random Forest'].predict(X_test)
# Exibir as previsões
#for seq_id, prediction in results.items():
#    print(f"ID: {seq_id}, Predição: {prediction}")

# Salvar as previsões em um arquivo
with open('previsionsRF.txt', 'w') as f:
    for seq_id, prediction in results.items():
        f.write(f"{seq_id}\t{prediction}\n")

# Baixar via navegador o arquivo de resultados
#from google.colab import files
#files.download('previsionsRF.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
## compara as predições com SVM e Randon forest
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
# Carregar as previsões do Random Forest
with open('previsionsRF.txt', 'r') as f:
    rf_predictions = {line.split('\t')[0]: line.split('\t')[1].strip() for line in f}

# Carregar as previsões do SVM otimizado
with open('PrevisionsSvmTun.txt', 'r') as f:
    svm_predictions = {line.split('\t')[0]: line.split('\t')[1].strip() for line in f}

# Verificar se as chaves (IDs) dos dicionários são iguais
assert set(rf_predictions.keys()) == set(svm_predictions.keys()), "Os IDs das previsões não coincidem"

# Criar um DataFrame para comparar as previsões
df_comparison = pd.DataFrame({
    'ID': list(rf_predictions.keys()),
    'RandomForest': list(rf_predictions.values()),
    'SVMTuned': list(svm_predictions.values())
})

# Calcular a acurácia entre as previsões dos dois modelos
accuracy = accuracy_score(list(rf_predictions.values()), list(svm_predictions.values()))
print(f"Acurácia entre as previsões de Random Forest e SVM otimizado: {accuracy}")

# Mostrar um relatório de classificação para comparar as previsões
print("\nRelatório de Classificação:")
print(classification_report(list(rf_predictions.values()), list(svm_predictions.values())))

# Exibir as previsões onde os modelos discordam
disagreement = df_comparison[df_comparison['RandomForest'] != df_comparison['SVMTuned']]
print("\nPrevisões onde os modelos discordam:")
print(disagreement)

# Exibir a contagem de concordâncias e discordâncias
print(f"\nTotal de previsões: {len(df_comparison)}")
print(f"Previsões concordantes: {len(df_comparison) - len(disagreement)}")
print(f"Previsões discordantes: {len(disagreement)}")

Acurácia entre as previsões de Random Forest e SVM otimizado: 1.0

Relatório de Classificação:
                 precision    recall  f1-score   support

 DNA-transposon       1.00      1.00      1.00        37
Retrotransposon       1.00      1.00      1.00       190

       accuracy                           1.00       227
      macro avg       1.00      1.00      1.00       227
   weighted avg       1.00      1.00      1.00       227


Previsões onde os modelos discordam:
Empty DataFrame
Columns: [ID, RandomForest, SVMTuned]
Index: []

Total de previsões: 227
Previsões concordantes: 227
Previsões discordantes: 0
